<a href="https://colab.research.google.com/github/kimhanui/2020_AI_study/blob/develop/%EB%8F%99%EB%AC%BC%EB%B6%84%EB%A5%98(%EC%84%A0%ED%98%95%EB%B6%84%EB%A5%98).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 데이터 로더

In [30]:
import numpy as np
import torch
import torch.optim as optim

# For reproductibility
torch.manual_seed(1)

! curl 'https://raw.githubusercontent.com/sejongresearch/2020.Spring.AI/master/HW/zoo_class.csv' -o 'zoo_class.csv'
print('Done.....\n')
xy = np.loadtxt('zoo_class.csv', delimiter=',', dtype=np.float32)

x_data = torch.FloatTensor(xy[:,0:-1])
y_data = torch.LongTensor(xy[:,[-1]])

print(x_data[:5])
print(y_data[:5])

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3535  100  3535    0     0  28975      0 --:--:-- --:--:-- --:--:-- 28975
Done.....

tensor([[1., 0., 0., 1., 0., 0., 1., 1., 1., 1., 0., 0., 4., 0., 0., 1.],
        [1., 0., 0., 1., 0., 0., 0., 1., 1., 1., 0., 0., 4., 1., 0., 1.],
        [0., 0., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 1., 0., 0.],
        [1., 0., 0., 1., 0., 0., 1., 1., 1., 1., 0., 0., 4., 0., 0., 1.],
        [1., 0., 0., 1., 0., 0., 1., 1., 1., 1., 0., 0., 4., 1., 0., 1.]])
tensor([[0],
        [0],
        [3],
        [0],
        [0]])


## 모델 학습

In [60]:
import torch.nn.functional as F # for softmax

#모델 초기화
nb_class = 7
nb_data = len(y_data)

W = torch.zeros((16, nb_class), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
#optimizer 설정
optimizer = optim.SGD([W,b],lr=0.1)
nb_epochs = 300000

for epoch in range(nb_epochs+1):
    #hypothesis <- softmax
    hypothesis = F.softmax(x_data.matmul(W)+b, dim=1)

    #cost
    y_one_hot = torch.zeros(nb_data, nb_class)
    
    y_one_hot.scatter_(1, y_data, 1)  # one_hot 주의!! y_data = coloumn vector, scatter=털어넣다..?
    cost = (y_one_hot * -torch.log(F.softmax(hypothesis, dim=1))).sum(dim=1).mean()

    #cost -> hypothesis 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    #로그 출력
    if epoch % 10000 == 1:
        print('Epoch: {:4d}/{} cost: {:.4}'.format(epoch, nb_epochs, cost.item(), W))


Epoch:    1/300000 cost: 1.941
Epoch: 10001/300000 cost: 1.296
Epoch: 20001/300000 cost: 1.285
Epoch: 30001/300000 cost: 1.211
Epoch: 40001/300000 cost: 1.208
Epoch: 50001/300000 cost: 1.206
Epoch: 60001/300000 cost: 1.203
Epoch: 70001/300000 cost: 1.171
Epoch: 80001/300000 cost: 1.169
Epoch: 90001/300000 cost: 1.168
Epoch: 100001/300000 cost: 1.168
Epoch: 110001/300000 cost: 1.167
Epoch: 120001/300000 cost: 1.167
Epoch: 130001/300000 cost: 1.167
Epoch: 140001/300000 cost: 1.167
Epoch: 150001/300000 cost: 1.167
Epoch: 160001/300000 cost: 1.167
Epoch: 170001/300000 cost: 1.167
Epoch: 180001/300000 cost: 1.166
Epoch: 190001/300000 cost: 1.166
Epoch: 200001/300000 cost: 1.166
Epoch: 210001/300000 cost: 1.166
Epoch: 220001/300000 cost: 1.166
Epoch: 230001/300000 cost: 1.166
Epoch: 240001/300000 cost: 1.166
Epoch: 250001/300000 cost: 1.166
Epoch: 260001/300000 cost: 1.166
Epoch: 270001/300000 cost: 1.166
Epoch: 280001/300000 cost: 1.166
Epoch: 290001/300000 cost: 1.166


## 정답 예측

In [72]:
hypothesis = F.softmax(x_data.matmul(W) +b, dim=1)
predict = torch.argmax(hypothesis, dim=1)

# print(hypothesis)
print(predict)
print(y_data.squeeze())
#정확도 계산
correct_prediction = predict.float() == y_data.squeeze()
print(correct_prediction)
accuracy = correct_prediction.sum().item() / len(correct_prediction)
print('The model has an accuracy of {:2.2f}% for the training set !'.format(accuracy * 100))

tensor([0, 0, 3, 0, 0, 0, 0, 3, 3, 0, 0, 1, 3, 6, 6, 6, 1, 0, 3, 0, 1, 1, 0, 1,
        5, 4, 4, 0, 0, 0, 5, 0, 0, 1, 3, 0, 0, 1, 3, 5, 5, 1, 5, 1, 0, 0, 6, 0,
        0, 0, 0, 5, 4, 6, 0, 0, 1, 1, 1, 1, 3, 3, 2, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        6, 3, 0, 0, 2, 6, 1, 1, 2, 6, 3, 1, 0, 6, 3, 1, 5, 4, 2, 2, 3, 0, 0, 1,
        0, 5, 0, 6, 1])
tensor([0, 0, 3, 0, 0, 0, 0, 3, 3, 0, 0, 1, 3, 6, 6, 6, 1, 0, 3, 0, 1, 1, 0, 1,
        5, 4, 4, 0, 0, 0, 5, 0, 0, 1, 3, 0, 0, 1, 3, 5, 5, 1, 5, 1, 0, 0, 6, 0,
        0, 0, 0, 5, 4, 6, 0, 0, 1, 1, 1, 1, 3, 3, 2, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        6, 3, 0, 0, 2, 6, 1, 1, 2, 6, 3, 1, 0, 6, 3, 1, 5, 4, 2, 2, 3, 0, 0, 1,
        0, 5, 0, 6, 1])
tensor([True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True